In [1]:
import folium # create maps
from sklearn.cluster import KMeans #cluster the neighbourhoods
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
neighbourhoods = pd.read_csv('Toronto_NBHD_lat_lng.csv', index_col=0)
neighbourhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Next, we have a look at the Toronto map, with the neighbourhoods superimposed on it.

In [3]:
# get the latitude and longitude of Toronto city
geolocator = Nominatim(user_agent="toro_explorer")
location = geolocator.geocode('Toronto city, Ontario')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.7394839, -79.369314.


In [4]:
# create a map of Toronto city
toro_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers of the neighbourhood and borough on top
for borough, neighbourhood, lat, lng in zip(neighbourhoods['Borough'], neighbourhoods['Neighbourhood'], neighbourhoods['Latitude'], neighbourhoods['Longitude']):
    label = "{}; {}".format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toro_map)

toro_map

#### Let's cluster the neighbourhoods by their most popular venues. 

In [5]:
CLIENT_ID = '******************************' # my Foursquare ID
CLIENT_SECRET = '*****************************' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the Foursquare API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
toronto_venues = getNearbyVenues(neighbourhoods['Neighbourhood'], neighbourhoods['Latitude'], neighbourhoods['Longitude'])
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [8]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


#### Let's create a one hot dataframe for all the venue categories, and then calculate their proportions among all the venue categories in each neighbourhood.

In [9]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean()
toronto_grouped.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
Neighbourhood,,,,,,,,,,,,,,,,,,,,,
"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.0,...,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0
Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0


#### Let's have a look at the most common venues in each neighbourhood.

In [10]:
def return_most_common_venues(row, num_top_venues):
    row_sorted = row.sort_values(ascending=False)
    
    return row_sorted.index.values[:num_top_venues]

In [11]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = []
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns, index=toronto_grouped.index)

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, :] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighbourhood,,,,,,,,,,
"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Hotel,Sushi Restaurant,Bakery,American Restaurant,Asian Restaurant
Agincourt,Sandwich Place,Lounge,Breakfast Spot,Clothing Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",Playground,Park,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",Grocery Store,Pharmacy,Coffee Shop,Sandwich Place,Discount Store,Fried Chicken Joint,Beer Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant
"Alderwood, Long Branch",Pizza Place,Pharmacy,Gym,Sandwich Place,Coffee Shop,Athletics & Sports,Skating Rink,Pub,Dog Run,Dim Sum Restaurant


#### Now, we cluster the neighbourhoods with similar venues together.

In [12]:
# after testing with different values for n_clusters(2, 3, 4, 5), I decided 2 would be the best number
kclusters = 2

kmeans = KMeans(init='k-means++', n_clusters=kclusters, n_init=10).fit(toronto_grouped)
kmeans.labels_

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int32)

In [13]:
# add the cluster labels to the neighbourhood most common venue dataframe
neighbourhoods_venues_sorted['Cluster Label'] = kmeans.labels_
neighbourhoods_venues_sorted.head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
Neighbourhood,,,,,,,,,,,
"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Hotel,Sushi Restaurant,Bakery,American Restaurant,Asian Restaurant,1
Agincourt,Sandwich Place,Lounge,Breakfast Spot,Clothing Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,1
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",Playground,Park,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,0
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",Grocery Store,Pharmacy,Coffee Shop,Sandwich Place,Discount Store,Fried Chicken Joint,Beer Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant,1
"Alderwood, Long Branch",Pizza Place,Pharmacy,Gym,Sandwich Place,Coffee Shop,Athletics & Sports,Skating Rink,Pub,Dog Run,Dim Sum Restaurant,1


In [14]:
# merge with the postcode, borough, latitude and longitude information
toronto_merged = pd.merge(neighbourhoods, neighbourhoods_venues_sorted, on='Neighbourhood')
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Stadium,1
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,History Museum,Bar,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,1
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Pizza Place,Electronics Store,Rental Car Location,Mexican Restaurant,Medical Center,Spa,Breakfast Spot,Eastern European Restaurant,Dumpling Restaurant,Diner,1
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,1
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Bakery,Hakka Restaurant,Bank,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Fried Chicken Joint,Yoga Studio,Donut Shop,Dog Run,1


In [15]:
# create a new Toronto map
toro_clusters_map = folium.Map(location=[latitude, longitude], zoom_start=10)

colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, name, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(name) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toro_clusters_map)
       
toro_clusters_map

#### Now we explore the most common venues in each clusters and see why they are clustered together.

In [16]:
neighbourhoods_venues_sorted[neighbourhoods_venues_sorted['Cluster Label']==0].head(5)

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
Neighbourhood,,,,,,,,,,,
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",Playground,Park,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,0
"CFB Toronto, Downsview East",Airport,Park,Bus Stop,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,0
Caledonia-Fairbanks,Park,Fast Food Restaurant,Market,Pharmacy,Women's Store,Gym,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,0
East Toronto,Park,Metro Station,Convenience Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,0
"Forest Hill North, Forest Hill West",Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant,0


In [17]:
neighbourhoods_venues_sorted[neighbourhoods_venues_sorted['Cluster Label']==1].head(5)

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
Neighbourhood,,,,,,,,,,,
"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Hotel,Sushi Restaurant,Bakery,American Restaurant,Asian Restaurant,1
Agincourt,Sandwich Place,Lounge,Breakfast Spot,Clothing Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,1
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",Grocery Store,Pharmacy,Coffee Shop,Sandwich Place,Discount Store,Fried Chicken Joint,Beer Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant,1
"Alderwood, Long Branch",Pizza Place,Pharmacy,Gym,Sandwich Place,Coffee Shop,Athletics & Sports,Skating Rink,Pub,Dog Run,Dim Sum Restaurant,1
"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pharmacy,Deli / Bodega,Bridal Shop,Shopping Mall,Fast Food Restaurant,Sandwich Place,Diner,Supermarket,Sushi Restaurant,1


#### As we can see from the data, the cluster 0 neighbourhoods feature restaurants, markets and pharmacies, while the cluster 1 neighbourhoods feature park and transportation.